<a href="https://colab.research.google.com/github/aitormorais/Qiskit/blob/main/qbit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 19.9 MB/s eta 0:00:00


In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
#from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
#from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
#service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import random

In [ ]:
from qiskit_aer import AerSimulator
from qiskit import QuantumRegister, QuantumCircuit,Aer, execute,ClassicalRegister

In [ ]:
def generate_hamiltonian_graph(num_nodes):
    """
    Genera un grafo Hamiltoniano no dirigido con un número dado de nodos y aristas adicionales aleatorias.

    Un grafo Hamiltoniano es un grafo que tiene un ciclo Hamiltoniano (un ciclo que visita cada nodo una vez).
    Esta función primero crea un ciclo que conecta todos los nodos, y luego agrega aristas adicionales de manera aleatoria.

    Parámetros
    ----------
    num_nodes : int
        El número de nodos para incluir en el grafo.

    Devoluciones
    ------------
    G : networkx.classes.graph.Graph
        Un objeto de grafo no dirigido de NetworkX con 'num_nodes' nodos, un ciclo Hamiltoniano,
        y aristas adicionales que se añaden aleatoriamente.

    Ejemplo
    -------
    >>> G = generate_hamiltonian_graph(5)
    >>> print(G.edges(data=True))
    [(0, 1, {'weight': 1.0}), (0, 4, {'weight': 1.0}), (1, 2, {'weight': 1.0}), 
    (2, 3, {'weight': 1.0}), (3, 4, {'weight': 1.0})]
    """
    # Crea un nuevo grafo no dirigido
    G = nx.Graph()

    # Agrega los nodos al grafo
    G.add_nodes_from(range(num_nodes))

    # Genera un camino Hamiltoniano
    for i in range(num_nodes - 1):
        G.add_edge(i, i + 1, weight=1.0)
    G.add_edge(num_nodes - 1, 0, weight=1.0)  # Cierra el ciclo para hacerlo Hamiltoniano

    # Agrega más aristas de manera aleatoria
    for i in range(num_nodes):
        for j in range(i + 2, num_nodes):
            if np.random.random() > 0.5:  # Añade la arista con una probabilidad del 50%
                G.add_edge(i, j, weight=1.0)

    return G


In [ ]:
def compute_cost(n, w, G):
    """
    Calcula el costo de cada posible configuración de un sistema dado representado por un grafo y retorna el costo máximo y su configuración correspondiente.

    Parámetros
    ----------
    n : int
        El número de nodos en el grafo.
    w : numpy.ndarray
        Matriz de pesos de las aristas en el grafo.
    G : networkx.classes.graph.Graph
        Un objeto de grafo de NetworkX.
    pos : dict
        Un diccionario que mapea cada nodo a su posición en el grafo.

    Devoluciones
    -----------
    cost_dict : dict
        Un diccionario que mapea cada posible configuración del sistema a su costo asociado.
    best_cost_brute : int
        El costo máximo encontrado entre todas las configuraciones posibles.

    Ejemplo
    -------
    >>> cost_dict, best_cost_brute = compute_cost(n, w, G, pos)
    >>> print(best_cost_brute)
    15
    >>> print(cost_dict)
    {(0, 0, 0): 0, (0, 0, 1): 5, ..., (1, 1, 1): 15}
    """
    best_cost_brute = 0
    cost_dict = {}  # Diccionario para almacenar los casos y sus costos

    for b in range(2**n):
        x = [int(t) for t in reversed(list(bin(b)[2:].zfill(n)))]
        cost = 0
        for i in range(n):
            for j in range(n):
                cost = cost + w[i, j] * x[i] * (1 - x[j])
        if best_cost_brute < cost:
            best_cost_brute = cost
            xbest_brute = x
        
        # Almacena el caso y el costo en el diccionario
        cost_dict[tuple(x)] = cost

    colors = ["r" if xbest_brute[i] == 0 else "c" for i in range(n)]
    #draw_graph(G, colors, pos)

    return cost_dict, best_cost_brute


In [ ]:
def compute_weight_matrix(G, n):
    """
    Calcula la matriz de pesos a partir de un grafo dado.

    Parámetros
    ----------
    G : networkx.classes.graph.Graph
        Un objeto de grafo de NetworkX.
    n : int
        El número de nodos en el grafo.

    Devoluciones
    -----------
    w : numpy.ndarray
        Una matriz de 'n' por 'n' que representa los pesos de las aristas en el grafo. 
        Cada elemento w[i, j] es el peso de la arista entre los nodos i y j.

    Ejemplo
    -------
    >>> w = compute_weight_matrix(G, 5)
    >>> print(w)
    [[0. 1. 1. 1. 1.]
     [1. 0. 1. 1. 1.]
     [1. 1. 0. 1. 1.]
     [1. 1. 1. 0. 1.]
     [1. 1. 1. 1. 0.]]
    """
    # Calculando la matriz de pesos a partir del grafo aleatorio
    w = np.zeros([n, n])
    for i in range(n):
        for j in range(n):
            temp = G.get_edge_data(i, j, default=0)
            if temp != 0:
                w[i, j] = temp["weight"]
    return w


In [ ]:
def draw_graph(G, colors, pos):
    """
    Dibuja un grafo de NetworkX con nodos de colores y etiquetas de aristas.

    Parámetros
    ----------
    G : networkx.classes.graph.Graph
        Un objeto de grafo de NetworkX.
    colors : list
        Una lista de colores para los nodos. Cada nodo se dibuja con el color correspondiente de la lista.
    pos : dict
        Un diccionario que mapea cada nodo a su posición en el grafo.

    Ejemplo
    -------
    >>> draw_graph(G, ['red', 'red', 'cyan', 'cyan', 'cyan'], pos)
    """
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors, node_size=600, alpha=0.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G, "peso")
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)


In [ ]:
def string_to_tuple(s):
    """
    Convierte una cadena de texto en una tupla, interpretando cada carácter como un número entero.

    Parámetros
    ----------
    s : str
        La cadena de texto a convertir. Se espera que cada carácter de la cadena sea un dígito decimal.

    Devoluciones
    -----------
    tuple
        Una tupla de enteros correspondiente a los caracteres de la cadena.

    Ejemplo
    -------
    >>> t = string_to_tuple("1234")
    >>> print(t)
    (1, 2, 3, 4)
    """
    return tuple(int(char) for char in s)



Qiskit


In [ ]:
def get_random_number(qc, qubit, reg):
    """
    Genera un número aleatorio (0 o 1) utilizando un circuito cuántico.

    Parámetros
    ----------
    qc : qiskit.circuit.quantumcircuit.QuantumCircuit
        El circuito cuántico en el que se realizará la operación.
    qubit : qiskit.circuit.quantumregister.QuantumRegister
        El registro cuántico que contiene los qubits en los que se realizará la operación.
    reg : qiskit.circuit.classicalregister.ClassicalRegister
        El registro clásico que se utilizará para la medición.

    Devoluciones
    -----------
    first_digit : int
        Un número aleatorio (0 o 1) generado a partir de la medida del estado del qubit.

    Ejemplo
    -------
    >>> qc = QuantumCircuit(1, 1)
    >>> qubit = QuantumRegister(1)
    >>> reg = ClassicalRegister(1)
    >>> random_number = get_random_number(qc, qubit, reg)
    >>> print(random_number)
    0
    """
    qc.h(qubit[0])
    qc.measure(qubit[0], reg[0])
    
    with qc.if_test((reg, 1)):
        qc.x(qubit[0])

    backend_sim = AerSimulator()
    reset_sim_job = backend_sim.run(qc, shots = 1)
    reset_sim_result = reset_sim_job.result()
    counts = reset_sim_result.get_counts(qc)

    key = list(counts.keys())[0]
    first_digit = int(key[0])  # Convierte el primer carácter de la clave a un entero
    return first_digit


In [ ]:
def mutate_bit(qr, qc, bit):
    """
    Aplica una operación NOT (X) a un bit específico de un registro cuántico.
    
    Parámetros
    ----------
    qr : QuantumRegister
        El registro cuántico en el que se realizará la mutación.
    qc : QuantumCircuit
        El circuito cuántico que contiene el registro.
    bit : int
        El índice del bit que se va a mutar.
        
    Devoluciones
    ------------
    No devuelve nada; la función modifica el circuito cuántico directamente.
    """
    qc.x(qr[bit])


def mutate_multi_bit(qr, qc, probabilidad):
    """
    Aplica una operación NOT (X) a cada bit en un registro cuántico con cierta probabilidad.
    
    Parámetros
    ----------
    qr : QuantumRegister
        El registro cuántico en el que se realizarán las mutaciones.
    qc : QuantumCircuit
        El circuito cuántico que contiene el registro.
    probabilidad : float
        La probabilidad de que cada bit sea mutado.
        
    Devoluciones
    ------------
    No devuelve nada; la función modifica el circuito cuántico directamente.
    """
    for i in range(len(qr)):
        if np.random.random() > probabilidad:
            mutate_bit(qr, qc, i)


def mutate_bit_random(qr, qc, probabilidad):#prob >0 and prob <1 a mas grande prob menos oportunidades de mutar
    """
    Aplica una operación NOT (X) a un bit aleatorio en un registro cuántico con cierta probabilidad.
    
    Parámetros
    ----------
    qr : QuantumRegister
        El registro cuántico en el que se realizará la mutación.
    qc : QuantumCircuit
        El circuito cuántico que contiene el registro.
    probabilidad : float
        La probabilidad de que el bit sea mutado.
        
    Devoluciones
    ------------
    No devuelve nada; la función modifica el circuito cuántico directamente.
    """
    if np.random.random() > probabilidad:
        qc.x(qr[np.random.randint(0, len(qr))])


def mutate_exchange(qr, qc, probabilidad):
    """
    Intercambia dos bits aleatorios en un registro cuántico con cierta probabilidad.
    
    Parámetros
    ----------
    qr : QuantumRegister
        El registro cuántico en el que se realizará el intercambio.
    qc : QuantumCircuit
        El circuito cuántico que contiene el registro.
    probabilidad : float
        La probabilidad de que se realice el intercambio.
        
    Devoluciones
    ------------
    No devuelve nada; la función modifica el circuito cuántico directamente.
    """
    if np.random.random() > probabilidad:
        numero0 = np.random.randint(0, len(qr))
        numero1 = np.random.randint(0, len(qr))
        if numero0 != numero1:
            qc.swap(qr[numero0], qr[numero1])






In [ ]:
def mutate_exchange(qr,qc,probabilidad):
   if np.random.random() > probabilidad:
    numero0 = np.random.randint(0,len(qr))
    numero1  = np.random.randint(0,len(qr))
    if numero0 != numero1:
      qc.swap(qr[numero0], qr[numero1])




In [ ]:
def puntos(qr):
    """
    Genera dos números enteros aleatorios diferentes dentro del rango del tamaño del registro cuántico.

    Parámetros
    ----------
    qr : QuantumRegister
        El registro cuántico que se utiliza para determinar el rango de los números generados.

    Devoluciones
    ------------
    tuple
        Una tupla de dos enteros. El primer elemento de la tupla es siempre menor que el segundo.

    Ejemplo
    -------
    >>> qr = QuantumRegister(5)
    >>> p = puntos(qr)
    >>> print(p)
    (2, 4)  # Los valores exactos pueden variar debido a la generación aleatoria
    """
    numero = np.random.randint(1, len(qr) - 1) 
    num = np.random.randint(1, len(qr) - 1)
    while numero == num:
        num = np.random.randint(1, len(qr) - 1)
    if num > numero:
        return numero, num
    else:
        return num, numero



In [ ]:
def escribir(qc, qreg, bin_string):
    """
    Escribe o 'resetea' un registro cuántico, basándose en una cadena de texto binaria.

    Parámetros
    ----------
    qc : QuantumCircuit
        El circuito cuántico que contiene el registro.
    qreg : QuantumRegister
        El registro cuántico que se va a escribir o resetear.
    bin_string : str
        La cadena de texto binaria que se utilizará para escribir o resetear el registro. Si se quiere escribir un
        número específico, esta cadena debe representar el número en binario. Si se quiere resetear el registro, esta
        cadena debe ser la representación binaria actual del estado del registro cuántico.

    Devoluciones
    ------------
    No devuelve nada; la función modifica el circuito cuántico directamente.

    Lanza
    -----
    ValueError
        Si el tamaño del QuantumRegister no es suficientemente grande para el string binario.

    Ejemplo
    -------
    >>> qr = QuantumRegister(5)
    >>> qc = QuantumCircuit(qr)
    >>> escribir(qc, qr, '10101')
    """
    if qreg.size < len(bin_string):
        raise ValueError("El QuantumRegister no es suficientemente grande para el string binario")

    bin_string = bin_string[::-1]

    for i, bit in enumerate(bin_string):
        if bit == '1':
            qc.x(i)


In [ ]:
def create_qc(number_quantum_reg,qubits,number_classic_reg,bits):
  """facilita el crear circuitos cuanticos si quieres poner ancillas aconesjo hacerlo manualmente partiendo desde aqui"""
  
  # Create quantum registers
  qregs = [QuantumRegister(qubits, name=f'q{i}') for i in range(number_quantum_reg)]
  ancilla = QuantumRegister(1, name='ancilla')

  # Create classical registers
  cregs = [ClassicalRegister(bits, name=f'c{i}') for i in range(number_classic_reg)]
  cr_ancilla = ClassicalRegister(1, name='cr_ancilla')

  # Create a quantum circuit
  return QuantumCircuit(*qregs, ancilla, *cregs, cr_ancilla)


In [ ]:
def lanzar_dinamico(circuito, backend):
    """
    Ejecuta un circuito cuántico en un backend específico y devuelve los conteos de resultados.

    Parámetros
    ----------
    circuito : QuantumCircuit
        El circuito cuántico que se va a ejecutar.
    backend : Backend
        El backend de Qiskit en el que se ejecutará el circuito.

    Devoluciones
    ------------
    dict
        Un diccionario que contiene los conteos de resultados de la ejecución del circuito. Las claves son las 
        cadenas de texto binarias que representan los resultados, y los valores son los números de veces que cada 
        resultado ocurrió.

    Ejemplo
    -------
    >>> qc = QuantumCircuit(1)
    >>> qc.h(0)
    >>> backend = Aer.get_backend('qasm_simulator')
    >>> counts = lanzar_dinamico(qc, backend)
    >>> print(counts)
    {'0': 512, '1': 512}
    """
    backend_sim = backend
    tp_circuit = transpile(circuito, backend=backend, optimization_level=0)
    reset_sim_job = backend_sim.run(tp_circuit, shots=1, dynamic=True)
    reset_sim_result = reset_sim_job.result()
    return reset_sim_result.get_counts()


In [ ]:
def valores_hijos(resultados):#se supone que solo tenemos dos creg, es decir dos hijos
    """
    Extrae dos cadenas de texto binarias de los resultados de una ejecución de un circuito cuántico.

    Parámetros
    ----------
    resultados : dict
        El diccionario que contiene los conteos de resultados de la ejecución del circuito. Se espera que las claves
        sean cadenas de texto binarias separadas por espacios.

    Devoluciones
    ------------
    tuple
        Una tupla de dos tuplas de enteros. Cada tupla de enteros corresponde a los dígitos de una de las cadenas de 
        texto binarias.

    Ejemplo
    -------
    >>> resultados = {'1 101 011': 1}
    >>> valores = valores_hijos(resultados)
    >>> print(valores)
    ((0, 1, 1), (1, 0, 1))
    """
    for key in resultados.keys():
        return string_to_tuple(key.split(' ')[2]), string_to_tuple(key.split(' ')[1])

In [ ]:
def tuple_to_string(t):
    """
    Convierte una tupla de enteros en una cadena de texto, interpretando cada número entero como un carácter.

    Parámetros
    ----------
    t : tuple
        La tupla de enteros a convertir. Se espera que cada elemento de la tupla sea un número entero.

    Devoluciones
    ------------
    str
        Una cadena de texto correspondiente a los números de la tupla.

    Ejemplo
    -------
    >>> s = tuple_to_string((1, 2, 3, 4))
    >>> print(s)
    '1234'
    """
    return ''.join(str(num) for num in t)


In [ ]:
def crear_hijo(base, hasta, maximo, qreg_padre, qreg_madre, qreg_hijo, qc, qreg_ancilla, cr_ancilla):
    """
    Crea un nuevo registro cuántico hijo a partir de dos registros cuánticos padres utilizando el algoritmo genético.

    Parámetros
    ----------
    base : int
        El índice en el que comienza la sección intermedia del hijo.
    hasta : int
        El índice en el que termina la sección intermedia del hijo.
    maximo : int
        El tamaño total del registro cuántico.
    qreg_padre : QuantumRegister
        El registro cuántico del primer progenitor.
    qreg_madre : QuantumRegister
        El registro cuántico del segundo progenitor.
    qreg_hijo : QuantumRegister
        El registro cuántico del hijo que se va a crear.
    qc : QuantumCircuit
        El circuito cuántico en el que se realizarán las operaciones.
    qreg_ancilla : QuantumRegister
        El registro cuántico de ancilla utilizado para las operaciones.
    cr_ancilla : ClassicalRegister
        El registro clásico de ancilla utilizado para las operaciones.

    Devoluciones
    ------------
    No devuelve nada; la función modifica el QuantumCircuit directamente.

    Ejemplo
    -------
    >>> qr_padre = QuantumRegister(5, 'padre')
    >>> qr_madre = QuantumRegister(5, 'madre')
    >>> qr_hijo = QuantumRegister(5, 'hijo')
    >>> qc = QuantumCircuit(qr_padre, qr_madre, qr_hijo)
    >>> qr_ancilla = QuantumRegister(1, 'ancilla')
    >>> cr_ancilla = ClassicalRegister(1, 'cr_ancilla')
    >>> qc.add_register(qr_ancilla, cr_ancilla)
    >>> base = 1
    >>> hasta = 3
    >>> maximo = 5
    >>> crear_hijo(base, hasta, maximo, qr_padre, qr_madre, qr_hijo, qc, qr_ancilla, cr_ancilla)
    """
    if get_random_number(qc, qreg_ancilla, cr_ancilla) == 0:#se genera un numero aleatorio que indicara si se usa al padre o a la madre
        padre_hijo(qc, qreg_padre, qreg_hijo, qreg_ancilla, cr_ancilla, 0, base)
    else:
        padre_hijo(qc, qreg_madre, qreg_hijo, qreg_ancilla, cr_ancilla, 0, base)

    padre_hijo(qc, qreg_padre, qreg_hijo, qreg_ancilla, cr_ancilla, base, hasta)#medio
    padre_hijo(qc, qreg_madre, qreg_hijo, qreg_ancilla, cr_ancilla, hasta, maximo)#fin


In [ ]:
def padre_hijo(circuit, qreg_padre, qreg_hijo, qreg_ancilla, cr_ancilla, base, hasta):
    """
    Realiza el cruce entre el registro cuántico de un progenitor y el registro cuántico de un hijo.

    Parámetros
    ----------
    circuit : QuantumCircuit
        El circuito cuántico en el que se realizarán las operaciones.
    qreg_padre : QuantumRegister
        El registro cuántico del progenitor.
    qreg_hijo : QuantumRegister
        El registro cuántico del hijo.
    qreg_ancilla : QuantumRegister
        El registro cuántico de ancilla utilizado para las operaciones.
    cr_ancilla : ClassicalRegister
        El registro clásico de ancilla utilizado para las operaciones.
    base : int
        El índice en el que comienza la sección a cruzar.
    hasta : int
        El índice en el que termina la sección a cruzar.

    Devoluciones
    ------------
    No devuelve nada; la función modifica el QuantumCircuit directamente.

    Ejemplo
    -------
    >>> qr_padre = QuantumRegister(5, 'padre')
    >>> qr_hijo = QuantumRegister(5, 'hijo')
    >>> qc = QuantumCircuit(qr_padre, qr_hijo)
    >>> qr_ancilla = QuantumRegister(1, 'ancilla')
    >>> cr_ancilla = ClassicalRegister(1, 'cr_ancilla')
    >>> qc.add_register(qr_ancilla, cr_ancilla)
    >>> base = 1
    >>> hasta = 3
    >>> padre_hijo(qc, qr_padre, qr_hijo, qr_ancilla, cr_ancilla, base, hasta)
    """
    for i in range(base, hasta):
        circuit.cx(qreg_padre[i], qreg_hijo[i])
        circuit.x(qreg_padre[i])
        circuit.x(qreg_hijo[i])
        circuit.ccx(qreg_padre[i], qreg_hijo[i], qreg_ancilla[0])
        circuit.x(qreg_padre[i])
        circuit.x(qreg_hijo[i])
        circuit.cx(qreg_ancilla[0], qreg_hijo[i])
        circuit.x(qreg_hijo[i])
        circuit.x(qreg_ancilla[0])
        circuit.ccx(qreg_padre[i], qreg_ancilla[0], qreg_hijo[i])
        circuit.ccx(qreg_padre[i], qreg_hijo[i], qreg_ancilla[0])
        circuit.ccx(qreg_padre[i], qreg_ancilla[0], qreg_hijo[i])
        circuit.measure(qreg_ancilla[0], cr_ancilla[0])
        with circuit.if_test((cr_ancilla[0], 1)):
            circuit.x(qreg_ancilla[0])


In [1]:
def qga(number_quantum_reg,qubits,number_classic_reg,bits,cost_dict,probabilidad,mejor,backend,inicial):

    """
    qga(number_quantum_reg, qubits, number_classic_reg, bits, cost_dict, probabilidad, mejor, backend)

    Algoritmo genético cuántico que realiza una búsqueda de soluciones en un espacio cuántico.

    Parámetros:
    ------------
    number_quantum_reg: int
        Número de registros cuánticos a utilizar en el circuito.

    qubits: int
        Número de qubits en cada registro cuántico.

    number_classic_reg: int
        Número de registros clásicos a utilizar en el circuito.

    bits: int
        Número de bits en cada registro clásico.

    cost_dict: dict
        Un diccionario con los costos asociados a cada posible solución.

    probabilidad: float
        Probabilidad utilizada para las operaciones de mutación.

    mejor: tuple
        La mejor solución conocida hasta el momento.

    backend: Provider
        El backend en el que se ejecutará el circuito.
    
    inicial: tuple
        La solución inicial desde donde el algoritmo comenzará la búsqueda.

    Devuelve:
    ----------
    result: tuple
        La mejor solución encontrada por el algoritmo.

    Ejemplo:
    ---------
    >>> number_quantum_reg = 4
    >>> qubits = 6
    >>> number_classic_reg = 2
    >>> bits = 6
    >>> cost_dict = {(0, 1, 0, 0, 0, 1): 5.0, ...}
    >>> probabilidad = 0.1
    >>> mejor = (1, 0, 1, 1, 0, 0)
    >>> backend = Aer.get_backend('qasm_simulator')
    >>> qga(number_quantum_reg, qubits, number_classic_reg, bits, cost_dict, probabilidad, mejor, backend)
    (1, 0, 1, 1, 0, 0)
    """


    #qc = create_qc(number_quantum_reg,qubits,number_classic_reg,bits)
    # Create quantum registers
    qregs = [QuantumRegister(qubits, name=f'q{i}') for i in range(number_quantum_reg)]
    ancilla = QuantumRegister(1, name='ancilla')

    # Create classical registers
    cregs = [ClassicalRegister(bits, name=f'c{i}') for i in range(number_classic_reg)]
    cr_ancilla = ClassicalRegister(1, name='cr_ancilla')
    qc = QuantumCircuit(*qregs, ancilla, *cregs, cr_ancilla)
    generation = 0 
    best = False
    #result = ''
    #case = (0, 1, 0, 0, 0, 1) cost = 5.0 emepzaremos con este reg como padre y el segundo reg en superposicion
    #inicial = (0, 1, 0, 0, 0, 1)
    result=inicial
    escribir(qc,qc.qregs[0],tuple_to_string(inicial))
    for i in range(qc.qregs[1].size):
        qc.h(qc.qregs[1][i])
    #crear_hijo(base,hasta,qubits,qc.qregs[0],qc.qregs[1],qc.qregs[2],qc,qc.ancilla,qc.cr_ancilla)#usando al padre
    #crear_hijo(base,hasta,qubits,qc.qregs[1],qc.qregs[0],qc.qregs[3],qc,qc.ancilla,qc.cr_ancilla)#usando a la madre
    while generation <10:
        print("Generation: ",generation)
        crear_hijo(*puntos(qc.qregs[0]),qubits,qc.qregs[0],qc.qregs[1],qc.qregs[2],qc,ancilla,cr_ancilla)#usando al padre qc.qregs[-2]  # -2 since ancilla is the second last element added to qregs
        crear_hijo(*puntos(qc.qregs[0]),qubits,qc.qregs[1],qc.qregs[0],qc.qregs[3],qc,ancilla,cr_ancilla)#usando a la madre
        for i in range(qubits):
            qc.measure(qc.qregs[2][i],qc.cregs[0][i])
            qc.measure(qc.qregs[3][i],qc.cregs[1][i])
        hijo0,hijo1 = valores_hijos(lanzar_dinamico(qc,backend))
        print("recien_crea2 hijo0: ",hijo0,"costo: ",cost_dict[hijo0])
        print("recien_crea2 hijo1: ",hijo1,"costo: ",cost_dict[hijo1])
        if cost_dict[hijo0] < cost_dict[hijo1] and cost_dict[inicial] < cost_dict[hijo1]:
            escribir(qc,qc.qregs[0],tuple_to_string(hijo1))#borramos la info del padre
            inicial = hijo1
            result = hijo1
            
        if cost_dict[hijo0] > cost_dict[hijo1] and cost_dict[inicial] < cost_dict[hijo0]:
            escribir(qc,qc.qregs[0],tuple_to_string(hijo0))#borramos la info del padre
            inicial = hijo0
            result = hijo0
            
        if  cost_dict[hijo0] == mejor:
            result = hijo0
            print("fin!!")
            best = True
            break
            
            #best = True
            
        if  cost_dict[hijo1] == mejor:
            result = hijo1
            print("fin!!")
            best = True
            break


        
        cual = np.random.randint(0, 4)#aleatorio cual de las 4 mutaciones
        functions = {
            0: lambda: mutate_bit(qc.qregs[0], qc, np.random.randint(0, len(qc.qregs[0]))),
            1: lambda: mutate_multi_bit(qc.qregs[0], qc, probabilidad),
            2: lambda: mutate_bit_random(qc.qregs[0], qc, probabilidad),
            3: lambda: mutate_exchange(qc.qregs[0], qc, probabilidad)
        }
        functions[cual]()
        if cost_dict[hijo0]>cost_dict[hijo1] and cost_dict[hijo0]> cost_dict[inicial]:
            if cost_dict[hijo0] == mejor:
                best = True
                break
            escribir(qc,qc.qregs[0],tuple_to_string(hijo0))#borramos la info del padre
            inicial = hijo0
            result = hijo0
        
        if cost_dict[hijo0]<cost_dict[hijo1] and cost_dict[hijo1]> cost_dict[inicial]:
            if cost_dict[hijo1] == mejor:
                best = True
                break
            
            escribir(qc,qc.qregs[0],tuple_to_string(hijo1))#borramos la info del padre
            inicial = hijo1
            result = hijo1
        print("mutaciones_hijo0: ",hijo0,"costo: ",cost_dict[hijo0])
        print("mutaciones_hijo1: ",hijo1,"costo: ",cost_dict[hijo1])

        print("padre: ",inicial)
        print("hijo0: ",hijo0)
        print("hijo1: ",hijo1)
        generation+=1
        print(generation)
    return result,best

EJECUTAR AQUI PARA LANZAR EL QGA SOLO HA SIDO PROBADO PARA 6 NODOS POR GRAFO

In [ ]:
n = 6 #numero de nodos
grafo = generate_hamiltonian_graph(n)

In [ ]:
w = compute_weight_matrix(grafo,n)

In [ ]:
cost_dict, best_cost_brute = compute_cost(n, w, grafo )

In [ ]:

for clave, valor in cost_dict.items():
    print(clave, valor)


(0, 0, 0, 0, 0, 0) 0.0
(1, 0, 0, 0, 0, 0) 4.0
(0, 1, 0, 0, 0, 0) 2.0
(1, 1, 0, 0, 0, 0) 4.0
(0, 0, 1, 0, 0, 0) 4.0
(1, 0, 1, 0, 0, 0) 6.0
(0, 1, 1, 0, 0, 0) 4.0
(1, 1, 1, 0, 0, 0) 4.0
(0, 0, 0, 1, 0, 0) 4.0
(1, 0, 0, 1, 0, 0) 6.0
(0, 1, 0, 1, 0, 0) 6.0
(1, 1, 0, 1, 0, 0) 6.0
(0, 0, 1, 1, 0, 0) 6.0
(1, 0, 1, 1, 0, 0) 6.0
(0, 1, 1, 1, 0, 0) 6.0
(1, 1, 1, 1, 0, 0) 4.0
(0, 0, 0, 0, 1, 0) 3.0
(1, 0, 0, 0, 1, 0) 7.0
(0, 1, 0, 0, 1, 0) 5.0
(1, 1, 0, 0, 1, 0) 7.0
(0, 0, 1, 0, 1, 0) 5.0
(1, 0, 1, 0, 1, 0) 7.0
(0, 1, 1, 0, 1, 0) 5.0
(1, 1, 1, 0, 1, 0) 5.0
(0, 0, 0, 1, 1, 0) 5.0
(1, 0, 0, 1, 1, 0) 7.0
(0, 1, 0, 1, 1, 0) 7.0
(1, 1, 0, 1, 1, 0) 7.0
(0, 0, 1, 1, 1, 0) 5.0
(1, 0, 1, 1, 1, 0) 5.0
(0, 1, 1, 1, 1, 0) 5.0
(1, 1, 1, 1, 1, 0) 3.0
(0, 0, 0, 0, 0, 1) 3.0
(1, 0, 0, 0, 0, 1) 5.0
(0, 1, 0, 0, 0, 1) 5.0
(1, 1, 0, 0, 0, 1) 5.0
(0, 0, 1, 0, 0, 1) 7.0
(1, 0, 1, 0, 0, 1) 7.0
(0, 1, 1, 0, 0, 1) 7.0
(1, 1, 1, 0, 0, 1) 5.0
(0, 0, 0, 1, 0, 1) 5.0
(1, 0, 0, 1, 0, 1) 5.0
(0, 1, 0, 1, 0, 1) 7.0
(1, 1, 0, 1

In [ ]:
suma = 0
for valor in cost_dict.values():
    suma = suma+valor
print(np.ceil(suma/len(cost_dict)))

In [ ]:
best_cost_brute

In [ ]:
cost_dict[(1, 0, 1, 1, 0, 1)]

In [ ]:
from qiskit import IBMQ

In [ ]:
#provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
#backend = provider.get_backend('ibm_sherbrooke')
backend = AerSimulator()

In [ ]:
result = qga(4,6,2,6,cost_dict,0.6,best_cost_brute,backend,(0, 0, 0, 1, 1, 1))

acabo de añadir el parametro inicial para que sea mas facil de usar

In [ ]:
result = qga(4,6,2,6,cost_dict,0.6,9,(1, 1, 0, 0, 0, 1))

AQUI BORRAR ABAJO


In [ ]:
def int_bit(numero, longitud):
    # Comprobamos si el número es demasiado grande para la cantidad de bits especificada.
    if numero > 2 ** longitud:
        raise ValueError(f"El número {numero} es demasiado grande para representarse con {longitud} bits")

    # Convertimos el número a binario con 'bin()', y lo convertimos a string con 'str()'.
    # La función 'bin()' retorna un string que comienza con '0b' para indicar que es un número binario,
    # así que utilizamos el slicing '[2:]' para eliminar estos caracteres.
    binary_str = str(bin(numero))[2:]

    # Ahora rellenamos con ceros a la izquierda hasta tener la longitud deseada usando 'zfill()'.
    return binary_str.zfill(longitud)




In [ ]:
def evaluar_resultados(qc,hijo1,hijo2,cr_hijo1,cr_hijo2,i,dic_qga,padre,madre,dic_mirar):
  """funcion que se encarga de 'obtener' los resultados"""
  binario = format(i,'06b')
  
  if dic_qga[binario]>dic_qga[padre]or dic_qga[binario]>dic_qga[madre]:
    if dic_qga[binario]>dic_qga[padre]:
      
      for qubit in padre:
        qc.reset(qubit)

      del dic_qga[padre]
      dic_qga[binario] = dic_mirar[binario]#funcio por implementar que buscara el valor en la lista
    if dic_qga[binario]>dic_qga[madre]:#faltaria asegurarse de no duplicar el padre y madre
      dic_qga[binario] = dic_mirar[binario]
      del dic_qga[madre]





In [ ]:
def get_keys_with_highest_values(d, n):
    sorted_items = sorted(d.items(), key=lambda item: item[1], reverse=True)
    top_n_items = sorted_items[:n]
    top_n_keys = [item[0] for item in top_n_items]
    return top_n_keys


ANCILLAS

OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
//x q[0];
//x q[1];
cx q[0], q[1];
ccx q[0], q[1], q[3];
ccx q[3], q[2], q[1];
ccx q[1], q[3], q[0];
ccx q[1], q[2], q[3];
cx q[1], q[2];
cx q[0], q[1];
cx q[1], q[2];
cx q[1], q[3];
ccx q[3], q[2], q[0];
cx q[1], q[2];
cx q[1], q[3];

In [ ]:
from qiskit import QuantumRegister, QuantumCircuit, Aer, execute, ClassicalRegister

In [ ]:
crear_hijo(base,hasta,len(qc),qr1,qr2,qr3,qc,ancilla,cr)

In [ ]:
for i in range(len(qr3)):
  qc.measure(qr3[i],cr_hijo[i])